In [ ]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoTokenizer,GPT2LMHeadModel, get_scheduler 
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm

from accelerate import Accelerator
accelerator = Accelerator()

num_epochs = 10
lr = 5e-5
batch_size = 8
save_loc = '/home/arjun/Documents/ModelSaves/GPT2Alpaca'

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
dataset = load_dataset("tatsu-lab/alpaca")
dataset = dataset['train']
dataset
dataset = dataset.select(range(100))
# model = GPT2LMHeadModel.from_pretrained("gpt2")
# tokenizer = AutoTokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
tokenizer = AutoTokenizer.from_pretrained(save_loc)
model =   GPT2LMHeadModel.from_pretrained(save_loc)
new_dataset = {'input_ids':[], 'attention_mask':[]}

for example in dataset:
    input_text = example['text'].replace('###','')   
    encoded_data = tokenizer('<|startoftext|>' + input_text + '<|endoftext|>',truncation=True, max_length=768, padding="max_length")
    new_dataset['input_ids'].append(encoded_data['input_ids'])
    new_dataset['attention_mask'].append(encoded_data['attention_mask'])

new_dataset = Dataset.from_dict(new_dataset)
new_dataset.set_format("torch")
dataloader = DataLoader(new_dataset,shuffle=True, batch_size=batch_size)

In [ ]:
optimizer = AdamW(model.parameters(), lr=lr)
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
model.resize_token_embeddings(len(tokenizer))
# model.to(device)


In [ ]:
model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, dataloader, lr_scheduler
)

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    **default_args,
)

In [ ]:
from accelerate import Accelerator
from torch.utils.data.dataloader import DataLoader

dataloader = DataLoader(ds, batch_size=training_args.per_device_train_batch_size)

if training_args.gradient_checkpointing:
    model.gradient_checkpointing_enable()

accelerator = Accelerator(fp16=training_args.fp16)
model, optimizer, dataloader = accelerator.prepare(model, adam_bnb_optim, dataloader)

model.train()
for step, batch in enumerate(dataloader, start=1):
    loss = model(**batch).loss
    loss = loss / training_args.gradient_accumulation_steps
    accelerator.backward(loss)
    if step % training_args.gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
# progress_bar = tqdm(range(num_training_steps-1),desc='Training', unit='steps')
# model.train()   
# ep = 0
# for epoch in range(num_epochs):
#     total_train_loss=0
#     for batch in dataloader:
#         batch_data = batch['input_ids'].to(device)
#         attention = batch['attention_mask'].to(device)
#         model.zero_grad()  
#         outputs = model(  batch_data,
#                           labels=batch_data, 
#                           attention_mask = attention,
#                           token_type_ids=None
#                         )

#         loss = outputs[0]     
#         batch_loss = loss.item()
#         total_train_loss += batch_loss
#         loss.backward()          
#         optimizer.step()         
#         lr_scheduler.step()     
#         optimizer.zero_grad()    
#         progress_bar.update(1)   
#     avg_train_loss = total_train_loss / len(dataloader)
#     ep += 1
#     print('Epoch:',ep,'Average training loss = ',avg_train_loss)


In [ ]:

question = 'Write a poem on cow'

prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
 Instruction:{question}
 Response: """
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)

sample_outputs = model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=100,
                                max_length=100,
                                max_new_tokens=200,
                                top_p=.95, 
                                num_return_sequences= 5,
                                temperature = .9,
                                )

for i, sample_output in enumerate(sample_outputs):
    ans = tokenizer.decode(sample_output, skip_special_tokens=True).split('Response: ')
    print("\n\n-------------------------------------------------------------------------------------------------------------------------------------------")
    try:        print(f'<-{i+1}-> {ans[1]}')
    except:
        print(f'<-{i+1}-> ___No response___')